In [1]:
import os
import pandas as pd
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm

os.chdir('/home/lewis/PhD/compounds-research')
from compounds_research.compound import utils
from compounds_research import settings
from compounds_research.compound.fetch_exchange_rates import make_exchange_rate_df

In [2]:
tokens = []
for k in settings.COMPOUND_TOKEN_TRANSLATOR:
    tokens.append(settings.COMPOUND_TOKEN_TRANSLATOR[k])

In [67]:
# Return dictionaries for where UIP holds and where it doesn't hold
regression_results = {}
regression_results['pair'] = []
regression_results['alpha'] = []
regression_results['beta'] = []
regression_results['r-squared'] = []
regression_results['p-value_alpha'] = []
regression_results['p-value_beta'] = []

UIP_holds = {}
UIP_not_holds = {}
considered_tokens = []
for quote_token in tokens:
    remaining_tokens = [i for i in tokens if i != quote_token]
    for base_token in remaining_tokens:
        try:
            df = utils.make_ex_rate_and_interest_rate_df(quote_token=quote_token, 
                                                        base_token=base_token, 
                                                        rate_type='borrow_rates', 
                                                        frequency='D', 
                                                        limit=1000)
        except:
            print('Error making dataframe for ' + str(quote_token) + ' to ' + str(base_token))
            continue
        
        df['i_rate_diff'] = df[quote_token] - df[base_token]
        df['ex_rate_diff'] = df[str(base_token)+ '_' + str(quote_token)].diff().shift(-1)
        df = df.dropna()
        model = OLS(endog=df['ex_rate_diff'], exog=sm.add_constant(df['i_rate_diff']))
        res = model.fit(cov_type='HC1')
        # print(res.summary())

        regression_results['pair'].append(base_token + '_' + quote_token)
        regression_results['pair'].append(' ')
        
        regression_results['alpha'].append(res.params[0])
        regression_results['alpha'].append('(' + str(res.bse[0]) + ')')

        regression_results['beta'].append(res.params[1])
        regression_results['beta'].append('(' + str(res.bse[1]) + ')')

        regression_results['r-squared'].append(res.rsquared)
        regression_results['r-squared'].append(' ')

        regression_results['p-value_alpha'].append(res.pvalues[0])
        regression_results['p-value_alpha'].append(' ')

        regression_results['p-value_beta'].append(res.pvalues[1])
        regression_results['p-value_beta'].append(' ')

        hypotheses = '(const = 0), (i_rate_diff = 1)'
        f_test = res.f_test(hypotheses)
        if f_test.pvalue >= 0.05:
            UIP_holds[str(base_token)+ '_' + str(quote_token)] = f_test.pvalue
        else:
            UIP_not_holds[str(base_token)+ '_' + str(quote_token)] = f_test.pvalue
            

In [68]:
df_results = pd.DataFrame(regression_results)
df_results


,pair,alpha,beta,r-squared,p-value_alpha,p-value_beta
0,bat_eth,-2.25243e-05,-0.00117198,0.0449313,0.00237359,0.00303519
1,,(7.411730987378997e-06),(0.00039538604688090255),,,
2,zrx_eth,-3.87492e-06,-0.000166627,0.00152883,0.436228,0.512824
3,,(4.976919834344088e-06),(0.0002546071147137115),,,
4,usdc_eth,-1.26448e-05,-0.000126306,0.000147175,0.863582,0.879029
5,,(7.35948432513304e-05),(0.000829868773927508),,,
6,dai_eth,-5.24251e-05,-0.00140998,0.00862633,0.193791,0.2772
7,,(4.034420946892928e-05),(0.0012975702204103582),,,
8,sai_eth,-2.71397e-05,-0.000255699,0.00081275,0.650838,0.596726
9,,(5.996418551768337e-05),(0.00048325886846476145),,,


In [27]:
UIP_holds
# Generally observe that eth_ currency may follow UIP and wbtc_ currency may follow UIP

{'wbtc_eth': array(0.9558718),
 'eth_bat': array(0.2642726),
 'dai_bat': array(0.09326267),
 'rep_bat': array(0.99914414),
 'wbtc_bat': array(0.8401058),
 'eth_zrx': array(0.60902307),
 'rep_zrx': array(0.43421688),
 'wbtc_zrx': array(0.74053023),
 'eth_usdc': array(0.94762153),
 'wbtc_usdc': array(0.90384715),
 'eth_dai': array(0.34658843),
 'rep_dai': array(0.556139),
 'wbtc_dai': array(0.9521816),
 'eth_sai': array(0.88473201),
 'wbtc_sai': array(0.55053974),
 'eth_rep': array(0.94644723),
 'wbtc_rep': array(0.89540767),
 'zrx_wbtc': array(0.77478855),
 'rep_wbtc': array(0.88617388)}

In [5]:
UIP_not_holds

{'bat_eth': array(0.),
 'zrx_eth': array(0.),
 'usdc_eth': array(0.),
 'dai_eth': array(0.),
 'sai_eth': array(0.),
 'rep_eth': array(1.61078753e-236),
 'zrx_bat': array(8.95506065e-06),
 'usdc_bat': array(1.35739397e-09),
 'sai_bat': array(5.0378133e-11),
 'bat_zrx': array(2.04970112e-07),
 'usdc_zrx': array(2.39535509e-10),
 'dai_zrx': array(0.0224437),
 'sai_zrx': array(3.26880838e-20),
 'bat_usdc': array(4.21580247e-279),
 'zrx_usdc': array(4.3286539e-277),
 'dai_usdc': array(2.84937572e-149),
 'sai_usdc': array(0.),
 'rep_usdc': array(0.0150402),
 'bat_dai': array(2.11534421e-63),
 'zrx_dai': array(3.54739531e-61),
 'usdc_dai': array(1.24391685e-151),
 'sai_dai': array(4.80295309e-34),
 'bat_sai': array(1.31892e-318),
 'zrx_sai': array(5.7e-322),
 'usdc_sai': array(0.),
 'dai_sai': array(4.23885699e-20),
 'rep_sai': array(0.00123133),
 'bat_rep': array(0.),
 'zrx_rep': array(0.),
 'usdc_rep': array(0.),
 'dai_rep': array(3.49733239e-107),
 'sai_rep': array(0.),
 'eth_wbtc': array(

In [33]:
base_token = 'rep'
quote_token = 'wbtc'
df = utils.make_ex_rate_and_interest_rate_df(quote_token=quote_token, 
                                                base_token=base_token, 
                                                rate_type='borrow_rates', 
                                                frequency='D', 
                                                limit=1000)

df['i_rate_diff'] = df[quote_token] - df[base_token]
df['ex_rate_diff'] = df[str(base_token)+ '_' + str(quote_token)].diff().shift(-1)
df = df.dropna()
model = OLS(endog=df['ex_rate_diff'], exog=sm.add_constant(df['i_rate_diff']))
res = model.fit(cov_type='HC1')
print(res.summary())
hypotheses = '(const = 0), (i_rate_diff = 1)'
f_test = res.f_test(hypotheses)
print(f_test)

OLS Regression Results                            
Dep. Variable:           ex_rate_diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.2305
Date:                Mon, 01 Jun 2020   Prob (F-statistic):              0.632
Time:                        14:39:32   Log-Likelihood:                -3352.0
No. Observations:                 323   AIC:                             6708.
Df Residuals:                     321   BIC:                             6716.
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const         190.5178    521.092      0.366      0.715    -830.803    1211.83

In [10]:
res

In [39]:
res.pvalues[1]

0.6311776863101952